# Applying CNN model on the spectogram of EEG 

In [1]:
import torch
import torch.nn as nn
import torch.cuda as cuda
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pdb
import pickle 
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from os import listdir
from os.path import isfile, join

import torch.nn.functional as F
from sklearn.decomposition import PCA

import random 
import pandas as pd 
import scipy.io as sio
import csv
import os
from dplf import * 

In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

ModuleNotFoundError: No module named 'fastai.transforms'

In [2]:
dataDir = "/home/erl.local/zeinab.schaefer/codes/dataDMSpect/DMdata2/"
npyDir = "/home/erl.local/zeinab.schaefer/codes/dataDMSpect/npySpectData/"
numSubjects = 17

In [3]:
# generate the name of the files 
# the form is FARAH_sub0XY_LS(1/4)_tr(1:4)
# 16 subjects with LS1 and LS4, subject 17 has only LS1
# LS1:Loud speaker to the left 
# LS4:Loud speaker to the right 
fns = []
labels = []
for subject in range(1,numSubjects+1):
    for spk in [1,4]:
        for trial in range(1,4+1):
            dt = []
            fn = 'FARAH_'+'sub0%0.2d'%subject+'_LS'+str(spk)+'_'+str(trial) + 'spec'
            fns.append(fn)

## save as .npy 

In [4]:
def read_by_name(fname, dtPath, opt):
    fpath = dtPath + fname 
    if opt:
        dt = sio.loadmat(fpath)['S']
        #dt = dt.transpose()
        return dt 
    else:
        dt = sio.loadmat(fpath)['y']
        dt = dt.transpose()
        return np.delete(dt, 29, axis = 1)

In [40]:
for fn in fns:
    dt = read_by_name(fn, dataDir, True)
    path = npyDir +fn+'.npy'
    np.save(path, dt)
    # pd.DataFrame(dt).to_csv(path)

In [5]:
# we give only the sujbect number and left or right speaker, and it generated the list of names
def gen_nameLists(subjectNumber, speakerSide):
    ssides = [1,4]
    if speakerSide not in ssides:
        print("speaker side is 1 or 4")
        #break
    
    namelists = []
    for trial in [1,2]: #range(1,5):
        fname = 'FARAH_'+'sub0%0.2d'%subjectNumber+'_LS'+str(speakerSide)+'_'+str(trial)+'spec'
        namelists.append(fname)
    return namelists

In [37]:
dict_labels = dict()
dict_electrode_labels = dict()
for subj in range(1,numSubjects+1):
    
    rspk = gen_nameLists(subj, speakerSide=1)
    lspk = gen_nameLists(subj, speakerSide=4)
    
    for name in rspk:
        dict_labels[name] = '+1'
        
    for name in lspk:
        dict_labels[name] = '-1'

with open(npyDir + 'dict.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in dict_labels.items():
        writer.writerow([key, value])

In [47]:
class DirectionalMicrophoneDataset(Dataset):
    def __init__(self,path,samples):
        # path: the path to the npy files 
        # subjects: the list of the subjects to be loaded    
        self.path = path 
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, indx):
        itempath = self.path + self.samples[indx]['fname'] +'.npy'
        spectMat = np.load(itempath).T # loads the data 
        signal_label = self.samples[indx]['label'] # the corresponding label 
        return {'signal': spectMat, 'label': signal_label}
     
class ToTensor(object):
    def __call__(self, sample):
        signal, label = sample['signal'], sample['label']
        dt = torch.from_numpy(signal.transpose())
        return {'signal': dt, 'label': label}
    
    
class TransformedDataset(Dataset):
    def __init__(self, dataset, *transforms):
        self.dataset = dataset
        self.transforms = transforms
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        sample = self.dataset[idx]
        for transform in self.transforms:
            sample = transform(sample)
        return sample


def sample_for(subject, spk, trial, label):
    fname = 'FARAH_'+'sub0%0.2d'%subject+'_LS'+str(spk)+'_'+str(trial)
    return {'fname': fname, 'label': label }

def create_dt_samples():
    train_samples = []
    valid_samples = []
    for subject in range(1,18):           
        for spk in [1, 4]:
            
            if spk == 1:
                label = 0 # Right    
            else:
                label = 1 # Left
            # randomly take three trials out of four for training
            # one trial for testing
            all_trials = range(1, 3) # range(1,5)
            train_trials = random.sample(all_trials, 1)
            valid_trials = np.setdiff1d(all_trials, train_trials)
        
            
            # list of validation names accordingly
            for trial in valid_trials:
                valid_samples.append(sample_for(subject, spk, trial, label))
            
            for trial in train_trials:
                train_samples.append(sample_for(subject, spk, trial, label))
    return (train_samples, valid_samples)

def all_samples():
    samples = []
    for subject in range(1,18):
        for spk in [1,4]:
            if spk == 1:
                label = 0
            else:
                label = 1
            for trial in [1,2]:#range(1,5):
                samples.append(sample_for(subject, spk, trial, label))
    return samples

def random_split(samples, train_percent = 80):
    random.shuffle(samples)
    train_limit = (len(samples) * train_percent) // 100
    return samples[:train_limit], samples[train_limit:]

In [48]:
train_samples, valid_samples = create_dt_samples()

In [50]:
train_samples, valid_samples = random_split(all_samples())

In [38]:
df  = pd.read_csv(npyDir+'dict.csv')
df.columns = ['subjectName', 'label']
df.to_csv(npyDir + 'dict2.csv')

In [41]:
df = pd.read_csv(npyDir+'dict2.csv')
df.head()

,Unnamed: 0,subjectName,label
0,0,FARAH_sub001_LS1_2spec,1
1,1,FARAH_sub001_LS4_1spec,-1
2,2,FARAH_sub001_LS4_2spec,-1
3,3,FARAH_sub002_LS1_1spec,1
4,4,FARAH_sub002_LS1_2spec,1


In [4]:
import fastai.vision
import os
import fastai

In [1]:
path = "/home/erl.local/zeinab.schaefer/codes/dataDMSpect/SpectImages/"

In [ ]:
data = ImageDataBunch.fro(path, ds_tfms=tfms, size=24);

In [5]:
ImageDataBunch

NameError: name 'ImageDataBunch' is not defined

In [50]:
data = fastai.vision.ImageDataBunch.from_df(npyDir, df, size=24)

In [51]:
data

FileNotFoundError: [Errno 2] No such file or directory: '/home/erl.local/zeinab.schaefer/codes/dataDMSpect/npySpectData/./0'